In [1]:
import h3
from bitarray import bitarray
from bitarray.util import ba2int, ba2hex
import toolz

def to_lower(h):
    d = h3.string_to_h3(h)
    b = '{0:064b}'.format(d)

    return -ba2int(bitarray(b)[-52:])


def child_seq(h):
    """ Return list of child choices.
    """
    d = h3.string_to_h3(h)
    b = '{0:064b}'.format(d)[-45:]
    
    g = toolz.partition(3, b)
    g = (''.join(e) for e in g)
    g = map(bitarray, g)
    g = map(ba2int, g)
    g = filter(lambda x: x != 7, g)
    g = list(g)
    
    return g
    
def descr(h):
    return h3.h3_get_base_cell(h), child_seq(h)


def is_child(p, c):
    r = h3.h3_get_resolution(p)
    return p == h3.h3_to_parent(c, r)

In [2]:
## single resolution

h = h3.geo_to_h3(0,0,2)

hexes = h3.h3_to_children(h, 4)

seq = list(sorted(hexes, key=to_lower, reverse=True))

In [3]:
def get_parent(seq, i):
    if i == 0 or h3.h3_get_resolution(seq[i-1]) == 0:
        return None
    else:
        return h3.h3_to_parent(seq[i-1])

In [4]:
def do_it2(hexes):
    seq = list(sorted(hexes, key=to_lower, reverse=True))
    N = len(seq)
    
    num_children = []
    
    i = 0 # dense to the left of i. stack
    j = 0 # to inspect to the right of j. stream
    
    while j < N:
        if seq[j] == 0:
            j += 1
            continue
        
        # pull off of upper stream. we don't have to put it back usually, can just peek.
        e = seq[j]
        j += 1
        p = h3.h3_to_parent(e)
        seq[i] = e # does it make sense to put this thing on the stack early?
        
        
        cur_parent = get_parent(seq, i)
        
        # start a new thread
        if cur_parent is None:
            num_children.append(1)
            i += 1
            continue
        
        # if its one down
        if cur_parent == p:
            num_children[-1] += 1
            i += 1
            if num_children[-1] == 7:
                num_children.pop()
                i -= 7
                j -= 1
                seq[j] = p
            continue
        
        # if its more than one down
        if is_child(cur_parent, e):
            num_children.append(1)
            i += 1
        else:
            # starts a new thread in the *next* iteration!
            # can we combine the "start a new thread" conditions?
            num_children = []
            j -= 1
            seq[j] = e
            
    
    return seq[:i]

In [5]:
do_it2(hexes)

['82754ffffffffff']